In [6]:
using Pkg
Pkg.activate(".")
 Pkg.rm.(keys(Pkg.project().dependencies))

  Activating new project at `C:\Users\asus4\Desktop\julia_notebokk`


Nothing[]

In [138]:
import Pkg
using Pkg
Pkg.add("ResumableFunctions")
Pkg.add("StatsBase")
Pkg.add("Statistics")
Pkg.add("DataFrames")
Pkg.add("FFTW")
Pkg.add("NaNMath")
Pkg.add("FITSIO")
Pkg.add("Intervals")
Pkg.add("ProgressBars")
Pkg.add("Plots")
import Pkg
Pkg.add(["Distributions", "Statistics" ,"StatsBase" ,"HDF5" ])
using Test
using FFTW, Distributions, Statistics, StatsBase, HDF5
using DataFrames
using ResumableFunctions, StatsBase, Statistics, DataFrames
using FFTW, NaNMath, FITSIO, Intervals
using ProgressBars: tqdm as show_progress

   Resolving package versions...
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Project.toml`
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Project.toml`
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Project.toml`
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Project.toml`
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Project.toml`
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Project.toml`
 to `C:\Users\asus4\Desktop\julia_note

In [139]:
using Pkg

Pkg.add([
    "Statistics",
    "Distributions",
    "Test",
    "ResumableFunctions",
    "StatsBase",  # Provides Histogram functionality
    "Tables",
    "AbstractFFTs"
])
using Statistics
using Distributions
using Test
using ResumableFunctions
using StatsBase
using Tables
using NaNMath
using AbstractFFTs
using DataFrames

   Resolving package versions...
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Project.toml`
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Manifest.toml`


In [140]:
@resumable function get_flux_iterable_from_segments(
    times::AbstractVector{<:Real}, 
    gti::AbstractMatrix{<:Real}, 
    segment_size::Real; 
    n_bin=nothing, fluxes=nothing, errors=nothing
)
    dt = nothing
    binned = !isnothing(fluxes)
    if binned
        dt = Statistics.median(diff(@view times[1:min(100, end)]))
    end
    fun = _which_segment_idx_fun(; binned, dt)

    for (s, e, idx0, idx1) in fun(times, gti, segment_size)
        if idx1 - idx0 < 2
            @yield nothing
            continue
        end
        
        if !binned
            event_times = @view times[idx0:idx1-1]
            cts = fit(Histogram, float.(event_times .- s), nbins=n_bin).weights
        else
            cts = float.(@view fluxes[idx0+1:idx1])
            if !isnothing(errors)
                cts = NamedTuple{(:counts, :errors)}((cts, @view errors[idx0+1:idx1]))
            end
        end
        
        @yield cts
    end
end

function compare_tables(table1::DataFrame, table2::DataFrame; rtol=0.001, discard=Vector{Symbol}())
    s_discard = Set(Symbol.(discard))
    test_result = true

    # Ensure both DataFrames have the same columns before comparison
    common_columns = intersect(propertynames(table1), propertynames(table2))
    common_columns = setdiff(common_columns, s_discard)  # Remove discarded columns

    for key in common_columns
        oe, oc = getproperty(table1, key), getproperty(table2, key)
        
        if eltype(oe) <: Number && eltype(oc) <: Number
            if !(≈(oe, oc, rtol=rtol))
                test_result = false
                @warn "Mismatch in column $key: Maximum difference $(maximum(abs.(oe .- oc)))"
                break
            end
        else
            if !(oe == oc)
                test_result = false
                @warn "Mismatch in column $key: Values do not match"
                break
            end
        end
    end

    @test test_result
end
@testset "test_fourier" begin
    dt = 1
    len = 100
    ctrate = 10000
    N = len ÷ dt
    dt = len / N
    times = sort(rand(Uniform(0, len), len * ctrate))
    gti = [[0 len];;]
    bins = LinRange(0, len, N + 1)
    counts = fit(Histogram, times, bins).weights
    errs = fill!(similar(counts), 1) * sqrt(ctrate)
    bin_times = (@view(bins[1:end-1]) + @view(bins[2:end])) / 2
    segment_size = 20.0
    times2 = sort(rand(Uniform(0, len), len * ctrate))
    counts2 = fit(Histogram, times2, bins).weights
    errs2 = fill!(similar(counts2), 1) * sqrt(ctrate)

    @test get_average_ctrate(times, gti, segment_size) == ctrate
    @test get_average_ctrate(bin_times, gti, segment_size; counts=counts) == ctrate

    @testset "test_fts_from_segments_cts_and_events_are_equal" begin
        N = round(Int, segment_size / dt)
        fts_evts = collect(get_flux_iterable_from_segments(times, gti, segment_size, n_bin=N))
        fts_cts = collect(get_flux_iterable_from_segments(
                bin_times, gti, segment_size, fluxes=counts))
        @test fts_evts == fts_cts
    end
end


Test Summary: | Pass  Total  Time
test_fourier  |    3      3  1.8s


Test.DefaultTestSet("test_fourier", Any[Test.DefaultTestSet("test_fts_from_segments_cts_and_events_are_equal", Any[], 1, false, false, true, 1.742275953879e9, 1.742275954778e9, false, "In[140]")], 2, false, false, true, 1.742275952941e9, 1.742275954778e9, false, "In[140]")

In [141]:
function error_on_averaged_cross_spectrum(
    cross_power::AbstractVector{<:Complex}, 
    seg_power::AbstractVector{<:Real}, 
    ref_power::AbstractVector{<:Real}, n_ave::Integer,
    seg_power_noise::Real, ref_power_noise::Real;
    common_ref::Bool=false)

    if n_ave < 30
        @warn "n_ave is below 30. Please note that the error bars 
        on the quantities derived from the cross spectrum 
        are only reliable for a large number of averaged 
        powers."
    end
    
    two_n_ave = 2 * n_ave
    if common_ref
        Gsq = real.(cross_power .* conj(cross_power))
        bsq = bias_term.(seg_power, ref_power, seg_power_noise, ref_power_noise, n_ave)
        frac = @. (Gsq - bsq) / (ref_power - ref_power_noise)
        power_over_2n = ref_power / two_n_ave

        dRe = dIm = dG = @. NaNMath.sqrt(power_over_2n * (seg_power - frac))
        dphi = @. NaNMath.sqrt(power_over_2n * (seg_power / (Gsq - bsq) -
                       1 / (ref_power - ref_power_noise)))
    else
        PrPs = ref_power .* seg_power
        dRe = @. NaNMath.sqrt((PrPs + real(cross_power) ^ 2 - imag(cross_power) ^ 2) / two_n_ave)
        dIm = @. NaNMath.sqrt((PrPs - real(cross_power) ^ 2 + imag(cross_power) ^ 2) / two_n_ave)
        gsq = raw_coherence.(cross_power, seg_power, ref_power, seg_power_noise, ref_power_noise, n_ave)
        dphi = @. NaNMath.sqrt((1 - gsq) / (2 * gsq * n_ave))
        dG = sqrt.(PrPs ./ n_ave)
    end

    return dRe, dIm, dphi, dG
end
@testset "test_fourier" begin
    dt = 1
    len = 100
    ctrate = 10000
    N = len ÷ dt
    dt = len / N
    times = sort(rand(Uniform(0, len), len * ctrate))
    gti = [[0 len];;]
    bins = LinRange(0, len, N + 1)
    counts = fit(Histogram, times, bins).weights
    errs = fill!(similar(counts), 1) * sqrt(ctrate)
    bin_times = (@view(bins[1:end-1]) + @view(bins[2:end])) / 2
    segment_size = 20.0
    times2 = sort(rand(Uniform(0, len), len * ctrate))
    counts2 = fit(Histogram, times2, bins).weights
    errs2 = fill!(similar(counts2), 1) * sqrt(ctrate)

    @test get_average_ctrate(times, gti, segment_size) == ctrate
    @test get_average_ctrate(bin_times, gti, segment_size; counts=counts) == ctrate



    @testset "test_error_on_averaged_cross_spectrum_low_nave" begin
        for common_ref in [true, false]
            @test_logs (:warn, r"n_ave is below 30.") error_on_averaged_cross_spectrum(
                [4 + 1.0im], [2], [4], 29, 2, 2; common_ref=common_ref)
        end
    end
end


Test Summary: | Pass  Total  Time
test_fourier  |    4      4  1.6s


Test.DefaultTestSet("test_fourier", Any[Test.DefaultTestSet("test_error_on_averaged_cross_spectrum_low_nave", Any[], 2, false, false, true, 1.742275955135e9, 1.742275956462e9, false, "In[141]")], 2, false, false, true, 1.742275954819e9, 1.742275956462e9, false, "In[141]")

In [142]:
function avg_pds_from_events(
    times::AbstractVector{<:Real}, gti::AbstractMatrix{<:Real},
    segment_size::Real, dt::Real; norm::String="frac",
    use_common_mean::Bool=true, silent::Bool=false, 
    fluxes=nothing, errors=nothing)
    
    if isnothing(segment_size)
        segment_size = maximum(gti) - minimum(gti)
    end
    n_bin = round(Int, segment_size / dt)
    dt = segment_size / n_bin

    flux_iterable = get_flux_iterable_from_segments(times, gti, segment_size;
                                                    n_bin=n_bin, fluxes=fluxes,
                                                    errors=errors)
    result = avg_pds_from_iterable(flux_iterable, dt, norm=norm,
                                  use_common_mean=use_common_mean,
                                  silent=silent)
    if isnothing(result)
        return nothing
    end
    
    return DataFrame(result)  # Ensure DataFrame output
end
@testset "test_fourier" begin
    dt = 1
    len = 100
    ctrate = 10000
    N = len ÷ dt
    dt = len / N
    times = sort(rand(Uniform(0, len), len * ctrate))
    gti = [[0 len];;]
    bins = LinRange(0, len, N + 1)
    counts = fit(Histogram, times, bins).weights
    errs = fill!(similar(counts), 1) * sqrt(ctrate)
    bin_times = (@view(bins[1:end-1]) + @view(bins[2:end])) / 2
    segment_size = 20.0
    times2 = sort(rand(Uniform(0, len), len * ctrate))
    counts2 = fit(Histogram, times2, bins).weights
    errs2 = fill!(similar(counts2), 1) * sqrt(ctrate)

    @test get_average_ctrate(times, gti, segment_size) == ctrate
    @test get_average_ctrate(bin_times, gti, segment_size; counts=counts) == ctrate



     @testset "test_avg_pds_bad_input" begin
        _times = rand(Uniform(0,1000),1)
        out_ev = avg_pds_from_events(_times, gti, segment_size, dt, silent=true)
        @test isnothing(out_ev)
    end
end

Test Summary: | Pass  Total  Time
test_fourier  |    3      3  0.3s


Test.DefaultTestSet("test_fourier", Any[Test.DefaultTestSet("test_avg_pds_bad_input", Any[], 1, false, false, true, 1.742275956803e9, 1.74227595684e9, false, "In[142]")], 2, false, false, true, 1.742275956496e9, 1.74227595684e9, false, "In[142]")

In [143]:
function avg_cs_from_iterables(
    flux_iterable1,
    flux_iterable2,
    dt::Real;
    norm::String="frac",
    use_common_mean::Bool=true,
    silent::Bool=false,
    fullspec::Bool=false,
    power_type::String="all",
    return_auxil::Bool=false)
    
    local_show_progress = show_progress
    if silent
        local_show_progress = (a) -> a
    end
    # Initialize stuff
    cross = unnorm_cross = unnorm_pds1 = unnorm_pds2 = pds1 = pds2 = nothing
    n_ave = 0
    fgt0 = n_bin = freq = nothing

    sum_of_photons1 = sum_of_photons2 = 0
    common_variance1 = common_variance2 = common_variance = nothing

    for (flux1, flux2) in local_show_progress(zip(flux_iterable1,
                                                flux_iterable2))
        if isnothing(flux1) || isnothing(flux2) || all(iszero, flux1.counts) || all(iszero, flux2.counts)
            continue
        end

        # Does the flux iterable return the uncertainty?
        # If so, define the variances
        variance1 = variance2 = nothing
        if hasproperty(flux1, :errors)
            err1 = flux1.errors
            variance1 = Statistics.mean(err1) ^ 2
        end
        if hasproperty(flux2, :errors)
            err2 = flux2.errors
            variance2 = Statistics.mean(err2) ^ 2
        end

        # Only use the variance if both flux iterables define it.
        if isnothing(variance1) || isnothing(variance2) 
            variance1 = variance2 = nothing
        else
            common_variance1 = sum_if_not_none_or_initialize(common_variance1,
                                                             variance1)
            common_variance2 = sum_if_not_none_or_initialize(common_variance2,
                                                             variance2)
        end

        n_bin = length(flux1.counts)

        # At the first loop, we define the frequency array and the range of
        # positive frequency bins (after the first loop, cross will not be
        # nothing anymore)
        if isnothing(cross)
            freq = fftfreq(n_bin, dt)
            fgt0 = positive_fft_bins(n_bin)
        end

        # Calculate the FFTs
        ft1 = fft(flux1.counts)
        ft2 = fft(flux2.counts)

        # Calculate the sum of each light curve, to calculate the mean
        n_ph1 = sum(flux1.counts)
        n_ph2 = sum(flux2.counts)
        n_ph = sqrt(n_ph1 * n_ph2)

        # Calculate the unnormalized cross spectrum
        unnorm_power = ft1 .* conj.(ft2)

        # If requested, calculate the auxiliary PDSs
        if return_auxil
            unnorm_pd1 = real.(ft1 .* conj.(ft1))
            unnorm_pd2 = real.(ft2 .* conj.(ft2))
        end

        # Accumulate the sum to calculate the total mean of the lc
        sum_of_photons1 += n_ph1
        sum_of_photons2 += n_ph2

        # Take only positive frequencies unless the user wants the full
        # spectrum
        if !(fullspec)
            keepat!(unnorm_power, fgt0)
            if return_auxil
                keepat!(unnorm_pd1, fgt0)
                keepat!(unnorm_pd2, fgt0)
            end
        end

        cs_seg = unnorm_power
        if return_auxil
            p1_seg = unnorm_pd1
            p2_seg = unnorm_pd2
        end

        # If normalization has to be done interval by interval, do it here.
        if !(use_common_mean)
            mean1 = n_ph1 / n_bin
            mean2 = n_ph2 / n_bin
            mean = n_ph / n_bin
            variance = nothing

            if !isnothing(variance1)
                variance = sqrt(variance1 * variance2)
            end

            cs_seg = normalize_periodograms(
                unnorm_power, dt, n_bin; mean_flux = mean, n_ph=n_ph, norm=norm,
                power_type=power_type, variance=variance
            )

            if return_auxil
                p1_seg = normalize_periodograms(
                    unnorm_pd1, dt, n_bin; mean_flux = mean1, n_ph=n_ph1, norm=norm,
                    power_type=power_type, variance=variance1
                )
                p2_seg = normalize_periodograms(
                    unnorm_pd2, dt, n_bin; mean_flux = mean2, n_ph=n_ph2, norm=norm,
                    power_type=power_type, variance=variance2
                )
            end
        end
        # Initialize or accumulate final averaged spectra
        cross = sum_if_not_none_or_initialize(cross, cs_seg)
        unnorm_cross = sum_if_not_none_or_initialize(unnorm_cross,
                                                     unnorm_power)

        if return_auxil
            unnorm_pds1 = sum_if_not_none_or_initialize(unnorm_pds1,
                                                        unnorm_pd1)
            unnorm_pds2 = sum_if_not_none_or_initialize(unnorm_pds2,
                                                        unnorm_pd2)
            pds1 = sum_if_not_none_or_initialize(pds1, p1_seg)
            pds2 = sum_if_not_none_or_initialize(pds2, p2_seg)
        end

        n_ave += 1
    end

    # If no valid intervals were found, return only `nothing`s
    if isnothing(cross)
        return nothing
    end

    # Calculate the mean number of photons per chunk
    n_ph1 = sum_of_photons1 / n_ave
    n_ph2 = sum_of_photons2 / n_ave
    n_ph = sqrt(n_ph1 * n_ph2)

    # Calculate the common mean number of photons per bin
    common_mean1 = n_ph1 / n_bin
    common_mean2 = n_ph2 / n_bin
    common_mean = n_ph / n_bin

    if !isnothing(common_variance1)
        # Note: the variances we summed were means, not sums. Hence M, not M*N
        common_variance1 /= n_ave
        common_variance2 /= n_ave
        common_variance = sqrt(common_variance1 * common_variance2)
    end

    # Transform the sums into averages
    cross ./= n_ave
    unnorm_cross ./= n_ave
    if return_auxil
        unnorm_pds1 ./= n_ave
        unnorm_pds2 ./= n_ave
    end

    # Finally, normalize the cross spectrum (only if not already done on an
    # interval-to-interval basis)
    if use_common_mean
        cross = normalize_periodograms(
            unnorm_cross,
            dt,
            n_bin;
            mean_flux = common_mean,
            n_ph=n_ph,
            norm=norm,
            variance=common_variance,
            power_type=power_type,
        )
        if return_auxil
            pds1 = normalize_periodograms(
                unnorm_pds1,
                dt,
                n_bin;
                mean_flux = common_mean1,
                n_ph=n_ph1,
                norm=norm,
                variance=common_variance1,
                power_type=power_type,
            )
            pds2 = normalize_periodograms(
                unnorm_pds2,
                dt,
                n_bin;
                mean_flux = common_mean2,
                n_ph=n_ph2,
                norm=norm,
                variance=common_variance2,
                power_type=power_type,
            )
        end
    end
    # If the user does not want negative frequencies, don't give them
    if !(fullspec)
        freq = freq[fgt0]
    end

    # Create DataFrame with the computed data
    results = DataFrame()
    results[!,"freq"] = freq
    results[!,"power"] = cross
    results[!,"unnorm_power"] = unnorm_cross

    if return_auxil
        results[!,"pds1"] = pds1
        results[!,"pds2"] = pds2
        results[!,"unnorm_pds1"] = unnorm_pds1
        results[!,"unnorm_pds2"] = unnorm_pds2
    end

    # Create NamedTuple with metadata instead of using attach_metadata
    metadata = (
        n = n_bin, 
        m = n_ave, 
        dt = dt,
        norm = norm,
        df = 1 / (dt * n_bin),
        segment_size = dt * n_bin,
        nphots = n_ph,
        nphots1 = n_ph1, 
        nphots2 = n_ph2,
        countrate1 = common_mean1 / dt,
        countrate2 = common_mean2 / dt,
        mean = common_mean,
        mean1 = common_mean1,
        mean2 = common_mean2,
        power_type = power_type,
        fullspec = fullspec,
        variance = common_variance,
        variance1 = common_variance1,
        variance2 = common_variance2
    )
    
    # Return both DataFrame and metadata as a tuple
    return (results, metadata)
end
function avg_cs_from_iterables_quick(flux_iterable1, flux_iterable2,
                                     dt::Real; norm::String="frac")
    unnorm_cross = unnorm_pds1 = unnorm_pds2 = nothing
    n_ave = 0
    fgt0 = n_bin = freq = nothing
    sum_of_photons1 = sum_of_photons2 = 0
    for (flux1, flux2) in zip(flux_iterable1, flux_iterable2)
        if isnothing(flux1) || isnothing(flux2) || all(iszero, flux1.counts) || all(iszero, flux2.counts)
            continue
        end
        n_bin = length(flux1.counts)
        # Calculate the sum of each light curve, to calculate the mean
        n_ph1 = sum(flux1.counts)
        n_ph2 = sum(flux2.counts)
        # At the first loop, we define the frequency array and the range of
        # positive frequency bins (after the first loop, cross will not be
        # nothing anymore)
        if isnothing(unnorm_cross)
            freq = fftfreq(n_bin, dt)
            fgt0 = positive_fft_bins(n_bin)
        end
        # Calculate the FFTs
        ft1 = fft(flux1.counts)
        ft2 = fft(flux2.counts)
        # Calculate the unnormalized cross spectrum
        unnorm_power = ft1 .* conj.(ft2)
        # Accumulate the sum to calculate the total mean of the lc
        sum_of_photons1 += n_ph1
        sum_of_photons2 += n_ph2
        # Take only positive frequencies
        keepat!(unnorm_power, fgt0)
        # Initialize or accumulate final averaged spectrum
        unnorm_cross = sum_if_not_none_or_initialize(unnorm_cross,
                                                     unnorm_power)
        n_ave += 1
    end
    # If no valid intervals were found, return only `nothing`s
    if isnothing(unnorm_cross)
        return nothing
    end
    # Calculate the mean number of photons per chunk
    n_ph1 = sum_of_photons1 / n_ave
    n_ph2 = sum_of_photons2 / n_ave
    n_ph = sqrt(n_ph1 * n_ph2)
    # Calculate the mean number of photons per bin
    common_mean1 = n_ph1 / n_bin
    common_mean2 = n_ph2 / n_bin
    common_mean = n_ph / n_bin
    # Transform the sums into averages
    unnorm_cross ./= n_ave
    # Finally, normalize the cross spectrum (only if not already done on an
    # interval-to-interval basis)
    cross = normalize_periodograms(
        unnorm_cross,
        dt,
        n_bin;
        mean_flux = common_mean,
        n_ph=n_ph,
        norm=norm,
        variance=nothing,
        power_type="all",
    )
    # No negative frequencies
    freq = freq[fgt0]
    
    # Create DataFrame with the computed data
    results = DataFrame()
    results[!,"freq"] = freq
    results[!,"power"] = cross
    results[!,"unnorm_power"] = unnorm_cross
    
    # Create NamedTuple with metadata instead of using attach_metadata
    metadata = (
        n = n_bin, 
        m = n_ave, 
        dt = dt,
        norm = norm,
        df = 1 / (dt * n_bin),
        nphots = n_ph,
        nphots1 = n_ph1, 
        nphots2 = n_ph2,
        variance = nothing,
        mean = common_mean,
        mean1 = common_mean1,
        mean2 = common_mean2,
        power_type = "all",
        fullspec = false,
        segment_size = dt * n_bin
    )
    
    # Return both DataFrame and metadata as a tuple
    return (results, metadata)
end
function avg_cs_from_events(
    times1:: AbstractVector{<:Real}, times2:: AbstractVector{<:Real},
    gti::AbstractMatrix{<:Real}, segment_size::Real, dt::Real;
    norm::String="frac", use_common_mean::Bool=true,
    fullspec::Bool=false, silent::Bool=false,
    power_type::String="all", fluxes1=nothing, fluxes2=nothing,
    errors1=nothing, errors2=nothing, return_auxil=false)
    
    if isnothing(segment_size) 
        segment_size = max(gti) - min(gti)
    end
    n_bin = round(Int, segment_size / dt)
    dt = segment_size / n_bin

    flux_iterable1 = get_flux_iterable_from_segments(
        times1, gti, segment_size; n_bin=n_bin, fluxes=fluxes1, errors=errors1
    )
    flux_iterable2 = get_flux_iterable_from_segments(
        times2, gti, segment_size; n_bin=n_bin, fluxes=fluxes2, errors=errors2
    )

    is_events = all(isnothing, (fluxes1, fluxes2, errors1, errors2))

    if (is_events && silent && use_common_mean && power_type == "all" && !fullspec && !return_auxil)
        results = avg_cs_from_iterables_quick(
            flux_iterable1,
            flux_iterable2,
            dt;
            norm=norm
        )
    else
        results = avg_cs_from_iterables(
            flux_iterable1,
            flux_iterable2,
            dt;
            norm=norm,
            use_common_mean=use_common_mean,
            silent=silent,
            fullspec=fullspec,
            power_type=power_type,
            return_auxil=return_auxil
        )
    end
    if isnothing(results)
        return nothing
    end
    return results[1]  # Return only the DataFrame, ignore metadata
end

@testset "test_fourier" begin
    dt = 1
    len = 100
    ctrate = 10000
    N = len ÷ dt
    dt = len / N
    times = sort(rand(Uniform(0, len), len * ctrate))
    gti = [[0 len];;]
    bins = LinRange(0, len, N + 1)
    counts = fit(Histogram, times, bins).weights
    errs = fill!(similar(counts), 1) * sqrt(ctrate)
    bin_times = (@view(bins[1:end-1]) + @view(bins[2:end])) / 2
    segment_size = 20.0
    times2 = sort(rand(Uniform(0, len), len * ctrate))
    counts2 = fit(Histogram, times2, bins).weights
    errs2 = fill!(similar(counts2), 1) * sqrt(ctrate)

    @test get_average_ctrate(times, gti, segment_size) == ctrate
    @test get_average_ctrate(bin_times, gti, segment_size; counts=counts) == ctrate



      @testset "test_avg_cs_bad_input" begin
        for return_auxil in [true, false]
            _times1 = rand(Uniform(0,1000),1)
            _times2 = rand(Uniform(0,1000),1)
            out_ev = avg_cs_from_events(_times1, _times2, gti,
                                        segment_size, dt, silent=true, return_auxil=return_auxil)
            @test isnothing(out_ev) 
        end
    end

end

Test Summary: | Pass  Total  Time
test_fourier  |    4      4  0.3s


Test.DefaultTestSet("test_fourier", Any[Test.DefaultTestSet("test_avg_cs_bad_input", Any[], 2, false, false, true, 1.742275957855e9, 1.742275957887e9, false, "In[143]")], 2, false, false, true, 1.742275957594e9, 1.742275957887e9, false, "In[143]")

In [144]:
function sum_if_not_none_or_initialize(current, new_value)
    if isnothing(current)
        if isa(new_value, AbstractArray)
            return copy(new_value)
        else
            return new_value
        end
    else
        if isa(current, AbstractArray) && isa(new_value, AbstractArray)
            return current .+ new_value
        else
            return current + new_value
        end
    end
end

@resumable function get_flux_iterable_from_segments(
    times::AbstractVector{<:Real}, 
    gti::AbstractMatrix{<:Real}, 
    segment_size::Real; 
    n_bin=nothing, fluxes=nothing, errors=nothing
)
    dt = nothing
    binned = !isnothing(fluxes)
    if binned
        dt = Statistics.median(diff(@view times[1:min(100, end)]))
    end
    fun = _which_segment_idx_fun(; binned, dt)

    for (s, e, idx0, idx1) in fun(times, gti, segment_size)
        if idx1 - idx0 < 2
            @yield nothing
            continue
        end
        
        if !binned
            event_times = @view times[idx0:idx1-1]
            cts = fit(Histogram, float.(event_times .- s), range(0, segment_size, length=n_bin+1)).weights
            @yield NamedTuple{(:counts,)}((cts,))
        else
            counts = float.(@view fluxes[idx0+1:idx1])
            if !isnothing(errors)
                errors_slice = float.(@view errors[idx0+1:idx1])
                if length(counts) == length(errors_slice)
                    @yield NamedTuple{(:counts, :errors)}((counts, errors_slice))
                else
                    @yield nothing
                end
            else
                @yield NamedTuple{(:counts,)}((counts,))
            end
        end
    end
end

function avg_pds_from_iterable(flux_iterable, dt::Real; norm::String="frac", 
                               use_common_mean::Bool=true,
                               silent::Bool=false)
    local_show_progress = identity
    if silent
        local_show_progress = identity
    end

    cross = unnorm_cross = nothing
    n_ave = 0
    freq = nothing

    sum_of_photons = 0
    common_variance = nothing
    fgt0 = nothing
    n_bin = nothing

    for flux_tuple in local_show_progress(flux_iterable)
        if isnothing(flux_tuple) || all(iszero, flux_tuple.counts)
            continue
        end

        flux = flux_tuple.counts
        variance = nothing
        if hasproperty(flux_tuple, :errors)
            err = flux_tuple.errors
            variance = Statistics.mean(err) ^ 2
        end

        n_bin = length(flux)
        ft = fft(flux)
        n_ph = sum(flux)
        unnorm_power = real.(ft .* conj.(ft))

        sum_of_photons += n_ph
        if !isnothing(variance)
            common_variance = sum_if_not_none_or_initialize(common_variance, variance)
        end

        if isnothing(cross)
            fgt0 = positive_fft_bins(n_bin)
            freq = fftfreq(n_bin, dt)[fgt0]
        end

        unnorm_power_filtered = unnorm_power[fgt0]
        
        cs_seg = unnorm_power_filtered
        if !(use_common_mean)
            mean = n_ph / n_bin
            cs_seg = normalize_periodograms(
                unnorm_power_filtered, dt, n_bin; mean_flux=mean, n_ph=n_ph,
                norm=norm, variance=variance,
            )
        end

        if isnothing(cross)
            cross = copy(cs_seg)
        else
            cross .+= cs_seg
        end
        
        if isnothing(unnorm_cross)
            unnorm_cross = copy(unnorm_power_filtered)
        else
            unnorm_cross .+= unnorm_power_filtered
        end
        
        n_ave += 1
    end

    if isnothing(cross)
        return nothing
    end

    n_ph = sum_of_photons / n_ave
    common_mean = n_ph / n_bin

    if !isnothing(common_variance)
        common_variance /= n_ave
    end

    unnorm_cross ./= n_ave
    cross ./= n_ave

    if use_common_mean
        cross = normalize_periodograms(
            unnorm_cross, dt, n_bin; mean_flux=common_mean, n_ph=n_ph,
            norm=norm, variance=common_variance
        )
    end

    results = DataFrame()
    results.freq = freq
    results.power = cross
    results.unnorm_power = unnorm_cross

    return results
end

function avg_pds_from_events(
    times::AbstractVector{<:Real}, gti::AbstractMatrix{<:Real},
    segment_size::Real, dt::Real; norm::String="frac",
    use_common_mean::Bool=true, silent::Bool=false, 
    fluxes=nothing, errors=nothing)
    
    if isnothing(segment_size)
        segment_size = maximum(gti) - minimum(gti)
    end
    n_bin = round(Int, segment_size / dt)
    dt = segment_size / n_bin

    flux_iterable = get_flux_iterable_from_segments(times, gti, segment_size;
                                                    n_bin=n_bin, fluxes=fluxes,
                                                    errors=errors)
    result = avg_pds_from_iterable(flux_iterable, dt, norm=norm,
                                  use_common_mean=use_common_mean,
                                  silent=silent)
    if isnothing(result)
        return nothing
    end
    
    return DataFrame(result)  # Ensure DataFrame output
end
@testset "test_fourier" begin
    dt = 1
    len = 100
    ctrate = 10000
    N = len ÷ dt
    dt = len / N
    times = sort(rand(Uniform(0, len), len * ctrate))
    gti = [[0 len];;]
    bins = LinRange(0, len, N + 1)
    counts = fit(Histogram, times, bins).weights
    errs = fill!(similar(counts), 1) * sqrt(ctrate)
    bin_times = (@view(bins[1:end-1]) + @view(bins[2:end])) / 2
    segment_size = 20.0
    times2 = sort(rand(Uniform(0, len), len * ctrate))
    counts2 = fit(Histogram, times2, bins).weights
    errs2 = fill!(similar(counts2), 1) * sqrt(ctrate)

    @test get_average_ctrate(times, gti, segment_size) == ctrate
    @test get_average_ctrate(bin_times, gti, segment_size; counts=counts) == ctrate
    @testset "test_avg_pds_use_common_mean_similar_stats" begin
        for norm in ["frac", "abs", "none", "leahy"]
            out_comm = avg_pds_from_events(
                times,
                gti,
                segment_size,
                dt,
                norm=norm,
                use_common_mean=true,
                silent=true,
                fluxes=nothing,
            )
            
            out = avg_pds_from_events(
                times,
                gti,
                segment_size,
                dt,
                norm=norm,
                use_common_mean=false,
                silent=true,
                fluxes=nothing,
            )
            
            @test !isnothing(out_comm) && !isnothing(out)
            @test std(out_comm.power) ≈ std(out.power) rtol=0.1
        end
    end


end

Test Summary: | Pass  Total  Time
test_fourier  |   10     10  0.8s


Test.DefaultTestSet("test_fourier", Any[Test.DefaultTestSet("test_avg_pds_use_common_mean_similar_stats", Any[], 8, false, false, true, 1.742275960113e9, 1.742275960519e9, false, "In[144]")], 2, false, false, true, 1.742275959757e9, 1.742275960519e9, false, "In[144]")

In [145]:
@testset "test_fourier" begin
    dt = 1
    len = 100
    ctrate = 10000
    N = len ÷ dt
    dt = len / N
    times = sort(rand(Uniform(0, len), len * ctrate))
    gti = [[0 len];;]
    bins = LinRange(0, len, N + 1)
    counts = fit(Histogram, times, bins).weights
    errs = fill!(similar(counts), 1) * sqrt(ctrate)
    bin_times = (@view(bins[1:end-1]) + @view(bins[2:end])) / 2
    segment_size = 20.0
    times2 = sort(rand(Uniform(0, len), len * ctrate))
    counts2 = fit(Histogram, times2, bins).weights
    errs2 = fill!(similar(counts2), 1) * sqrt(ctrate)

    @testset "test_avg_cs_use_common_mean_similar_stats" begin
        for norm in ["frac", "abs", "none", "leahy"], return_auxil in [true, false], fullspec in [true, false]
            out_comm = avg_cs_from_events(times, times2, gti, segment_size, dt, norm=norm, use_common_mean=true, silent=true, fullspec=fullspec, return_auxil=return_auxil)[!, :power]
            out = avg_cs_from_events(times, times2, gti, segment_size, dt, norm=norm, use_common_mean=false, silent=true, fullspec=fullspec, return_auxil=return_auxil)[!, :power]
            @test std(out_comm) ≈ std(out) rtol=0.1
        end
    end

    @testset "test_avg_pds_cts_and_events_are_equal" begin
        for use_common_mean in [true, false], norm in ["frac", "abs", "none", "leahy"]
            out_ev = avg_pds_from_events(times, gti, segment_size, dt, norm=norm, use_common_mean=use_common_mean, silent=true, fluxes=nothing)
            out_ct = avg_pds_from_events(bin_times, gti, segment_size, dt, norm=norm, use_common_mean=use_common_mean, silent=true, fluxes=counts)
            compare_tables(out_ev, out_ct)
        end
    end
    @testset "test_avg_pds_cts_and_err_and_events_are_equal" begin
        for use_common_mean in [true, false], norm in ["frac", "abs", "none", "leahy"]
            out_ev = avg_pds_from_events(times, gti, segment_size, dt, norm=norm, use_common_mean=use_common_mean, silent=true, fluxes=nothing)
            out_ct = avg_pds_from_events(bin_times, gti, segment_size, dt, norm=norm, use_common_mean=use_common_mean, silent=true, fluxes=counts, errors=errs)
            if use_common_mean
                compare_tables(out_ev, out_ct, rtol=0.01, discard=["variance"])
            else
                compare_tables(out_ev, out_ct, rtol=0.1, discard=["variance"])
            end
        end
    end
    @testset "test_avg_cs_cts_and_events_are_equal" begin
        for use_common_mean in [true, false], norm in ["frac", "abs", "none", "leahy"]
            out_ev = avg_cs_from_events(times, times2, gti, segment_size, dt, norm=norm, use_common_mean=use_common_mean, silent=true)
            out_ct = avg_cs_from_events(bin_times, bin_times, gti, segment_size, dt, norm=norm, use_common_mean=use_common_mean, silent=true, fluxes1=counts, fluxes2=counts2)
            if use_common_mean
                compare_tables(out_ev, out_ct, rtol=0.01)
            else
                compare_tables(out_ev, out_ct, rtol=0.1)
            end
        end
    end
    @testset "test_avg_cs_cts_and_err_and_events_are_equal" begin
        for use_common_mean in [true, false], norm in ["frac", "abs", "none", "leahy"]
            out_ev = avg_cs_from_events(times, times2, gti, segment_size, dt, norm=norm, use_common_mean=use_common_mean, silent=true)
            out_ct = avg_cs_from_events(bin_times, bin_times, gti, segment_size, dt, norm=norm, use_common_mean=use_common_mean, silent=true, fluxes1=counts, fluxes2=counts2, errors1=errs, errors2=errs2)
            discard = [m for m in propertynames(out_ev) if m == :variance]
            if use_common_mean
                compare_tables(out_ev, out_ct, rtol=0.01, discard=discard)
            else
                compare_tables(out_ev, out_ct, rtol=0.1, discard=discard)
            end
        end
    end
end


Test Summary: | Pass  Total   Time
test_fourier  |   48     48  10.7s


Test.DefaultTestSet("test_fourier", Any[Test.DefaultTestSet("test_avg_cs_use_common_mean_similar_stats", Any[], 16, false, false, true, 1.742275960885e9, 1.742275965611e9, false, "In[145]"), Test.DefaultTestSet("test_avg_pds_cts_and_events_are_equal", Any[], 8, false, false, true, 1.742275965611e9, 1.742275967078e9, false, "In[145]"), Test.DefaultTestSet("test_avg_pds_cts_and_err_and_events_are_equal", Any[], 8, false, false, true, 1.742275967078e9, 1.742275968733e9, false, "In[145]"), Test.DefaultTestSet("test_avg_cs_cts_and_events_are_equal", Any[], 8, false, false, true, 1.742275968733e9, 1.742275969882e9, false, "In[145]"), Test.DefaultTestSet("test_avg_cs_cts_and_err_and_events_are_equal", Any[], 8, false, false, true, 1.742275969882e9, 1.742275971323e9, false, "In[145]")], 0, false, false, true, 1.742275960582e9, 1.742275971323e9, false, "In[145]")

In [151]:
function normalize_frac(unnorm_power::AbstractVector{<:Number}, dt::Real, n_bin::Integer, 
                        mean_flux::Real; background_flux::Real=0.0)
    if background_flux > 0
        power = @. unnorm_power * 2 * dt / ((mean_flux - background_flux) ^ 2 *
                                          n_bin)
    else
        power = @. unnorm_power * 2 * dt / (mean_flux ^ 2 * n_bin)
    end
    return power
end

normalize_abs(unnorm_power::AbstractVector{<:Number}, dt::Real, n_bin::Integer) = 
    @. unnorm_power * 2 / n_bin / dt

normalize_leahy_from_variance(unnorm_power::AbstractVector{<:Number}, 
                              variance::Real, n_bin::Integer) = 
    @. unnorm_power * 2 / (variance * n_bin)

normalize_leahy_poisson(unnorm_power::AbstractVector{<:Number}, n_ph::Real) = 
    @. unnorm_power * 2 / n_ph

function normalize_periodograms(unnorm_power::AbstractVector{<:Number}, dt::Real, 
                                n_bin::Integer; mean_flux=nothing, n_ph=nothing,
                                variance=nothing, background_flux::Real=0.0, 
                                norm::String="frac",power_type::String="all")
    
    if norm == "leahy" && !isnothing(variance)
        pds = normalize_leahy_from_variance(unnorm_power, variance, n_bin)
    elseif norm == "leahy"
        pds = normalize_leahy_poisson(unnorm_power, n_ph)
    elseif norm == "frac"
        pds = normalize_frac(
            unnorm_power, dt, n_bin, mean_flux,
            background_flux=background_flux)
    elseif norm == "abs"
        pds = normalize_abs(unnorm_power, dt, n_bin)
    elseif norm == "none"
        pds = unnorm_power
    else 
        throw(ArgumentError("Unknown value for norm: $norm"))
    end

    if power_type == "all"
        return pds
    elseif power_type == "real"
        return real(pds)
    elseif power_type in ["abs", "absolute"]
        return abs.(pds)
    else 
        throw(ArgumentError("Unknown value for power_type: $power_type"))
    end

end

@testset "test_norm" begin
    mean = var = 100000
    N = 1000000
    dt = 0.2
    meanrate = mean / dt
    lc = rand(Poisson(mean), N)
    pds = abs2.(fft(lc))
    freq = fftfreq(N, dt)
    good = 2:(N ÷ 2)

    pdsabs = normalize_abs(pds, dt, size(lc, 1))
    pdsfrac = normalize_frac(pds, dt, size(lc, 1), mean)
    pois_abs = poisson_level("abs", meanrate=meanrate)
    pois_frac = poisson_level("frac", meanrate=meanrate)

    @test Statistics.mean(view(pdsabs, good)) ≈ pois_abs rtol=0.02
    @test Statistics.mean(view(pdsfrac, good)) ≈ pois_frac rtol=0.02

    mean = var = 100000.0
    N = 800000
    dt = 0.2
    df = 1 / (N * dt)
    freq = fftfreq(N, dt)
    good = freq .> 0
    meanrate = mean / dt
    lc = rand(Poisson(mean), N)
    nph = sum(lc)
    pds = view(abs2.(fft(lc)), good)
    lc_bksub = lc .- mean
    pds_bksub = view(abs2.(fft(lc_bksub)), good)
    lc_renorm = lc / mean
    pds_renorm = view(abs2.(fft(lc_renorm)), good)
    lc_renorm_bksub = lc_renorm .- 1
    pds_renorm_bksub = view(abs2.(fft(lc_renorm_bksub)), good)

    @testset "test_leahy_bksub_var_vs_standard" begin
        leahyvar = normalize_leahy_from_variance(pds_bksub, Statistics.var(lc_bksub), N)
        leahy = 2 .* pds ./ sum(lc)
        ratio = Statistics.mean(leahyvar ./ leahy)
        @test ratio ≈ 1 rtol=0.01
    end

    @testset "test_abs_bksub" begin
        ratio = normalize_abs(pds_bksub, dt, N) ./ normalize_abs(pds, dt, N)
        @test Statistics.mean(ratio) ≈ 1 rtol=0.01
    end

    @testset "test_frac_renorm_constant" begin
        ratio = normalize_frac(pds_renorm, dt, N, 1) ./ normalize_frac(pds, dt, N, mean)
        @test Statistics.mean(ratio) ≈ 1 rtol=0.01
    end

    @testset "test_frac_to_abs_ctratesq" begin
        ratio = (
            normalize_frac(pds, dt, N, mean) ./ normalize_abs(pds, dt, N) .* meanrate .^ 2
        )
        @test Statistics.mean(ratio) ≈ 1 rtol=0.01
    end

    @testset "test_total_variance" begin
        vdk_total_variance = sum((lc .- mean) .^ 2)
        ratio = Statistics.mean(pds) / vdk_total_variance
        @test Statistics.mean(ratio) ≈ 1 rtol=0.01
    end

    @testset "test_poisson_level_$(norm)" for norm in ["abs", "frac", "leahy", "none"]
        pdsnorm = normalize_periodograms(pds, dt, N; mean_flux=mean, n_ph=nph, norm=norm)
        @test Statistics.mean(pdsnorm) ≈ poisson_level(norm; meanrate=meanrate, n_ph=nph) rtol=0.01
    end

    @testset "test_poisson_level_real_$(norm)" for norm in ["abs", "frac", "leahy", "none"]
        pdsnorm = normalize_periodograms(pds, dt, N; mean_flux=mean, n_ph=nph, norm=norm, power_type="real")
        @test Statistics.mean(pdsnorm) ≈ poisson_level(norm; meanrate=meanrate, n_ph=nph) rtol=0.01
    end

    @testset "test_poisson_level_absolute_$(norm)" for norm in ["abs", "frac", "leahy", "none"]
        pdsnorm = normalize_periodograms(pds, dt, N; mean_flux=mean, n_ph=nph, norm=norm, power_type="abs")
        @test Statistics.mean(pdsnorm) ≈ poisson_level(norm; meanrate=meanrate, n_ph=nph) rtol=0.01
    end

    @testset "test_normalize_with_variance" begin
        pdsnorm = normalize_periodograms(pds, dt, N; mean_flux=mean, variance=var, norm="leahy")
        @test Statistics.mean(pdsnorm) ≈ 2 rtol=0.01
    end

    @testset "test_normalize_none" begin
        pdsnorm = normalize_periodograms(pds, dt, N; mean_flux=mean, n_ph=nph, norm="none")
        @test Statistics.mean(pdsnorm) ≈ Statistics.mean(pds) rtol=0.01
    end
end

Test Summary: | Pass  Total  Time
test_norm     |   21     21  1.8s


Test.DefaultTestSet("test_norm", Any[Test.DefaultTestSet("test_leahy_bksub_var_vs_standard", Any[], 1, false, false, true, 1.742277957556e9, 1.742277958172e9, false, "In[151]"), Test.DefaultTestSet("test_abs_bksub", Any[], 1, false, false, true, 1.742277958172e9, 1.742277958184e9, false, "In[151]"), Test.DefaultTestSet("test_frac_renorm_constant", Any[], 1, false, false, true, 1.742277958184e9, 1.742277958193e9, false, "In[151]"), Test.DefaultTestSet("test_frac_to_abs_ctratesq", Any[], 1, false, false, true, 1.742277958193e9, 1.742277958765e9, false, "In[151]"), Test.DefaultTestSet("test_total_variance", Any[], 1, false, false, true, 1.742277958765e9, 1.742277958771e9, false, "In[151]"), Test.DefaultTestSet("test_poisson_level_abs", Any[], 1, false, false, true, 1.742277958771e9, 1.742277958775e9, false, "In[151]"), Test.DefaultTestSet("test_poisson_level_frac", Any[], 1, false, false, true, 1.742277958775e9, 1.742277958778e9, false, "In[151]"), Test.DefaultTestSet("test_poisson_level_

In [147]:
# APPLYING GTI.JL FOR TESTING IT GENRATES OUTPUT OR NOT 

In [148]:
function get_total_gti_length(gti::AbstractMatrix{<:Real}; minlen::Real=0.0)
    lengths = diff(gti; dims =2)
    return sum(x->x > minlen ? x : zero(x), lengths)
end

function load_gtis(fits_file::String, gtistring::String="GTI")
    gti = FITS(fits_file) do lchdulist
        gtihdu = lchdulist[gtistring]
        get_gti_from_hdu(gtihdu)
    end
    return gti
end

function get_gti_from_hdu(gtihdu::TableHDU)

    if "START" in FITSIO.colnames(gtihdu)
        startstr = "START"
        stopstr = "STOP"
    else
        startstr = "Start"
        stopstr = "Stop"
    end

    gtistart = read(gtihdu,startstr)
    gtistop = read(gtihdu,stopstr)

    return mapreduce(permutedims, vcat, 
    [[a, b] for (a,b) in zip(gtistart, gtistop)])
end

function check_gtis(gti::AbstractMatrix)

    if ndims(gti) != 2 || size(gti,2) != 2
        throw(ArgumentError("Please check the formatting of the GTIs. 
       They need to be provided as [[gti00 gti01]; [gti10 gti11]; ...]."))
    end

    gti_start = @view gti[:, 1]
    gti_end = @view gti[:, 2]

    if any(gti_end < gti_start)
        throw(ArgumentError(
            "The GTI end times must be larger than the GTI start times."
        )) 
    end

    if any(@view(gti_start[begin+1:end]) < @view(gti_end[begin:end-1]))
        throw(ArgumentError(
            "This GTI has overlaps"
        ))
    end
end

function create_gti_mask(times::AbstractVector{<:Real},gtis::AbstractMatrix{<:Real};
                         safe_interval::AbstractVector{<:Real}=[0,0], min_length::Real=0,
                         dt::Real = -1, epsilon::Real = 0.001)

    if isempty(times)
        throw(ArgumentError("Passing an empty time array to create_gti_mask"))
    end

    check_gtis(gtis)
    mask = zeros(Bool,length(times))

    if min_length>0
        gtis = gtis[min_length .< @view(gtis[:,2]) - @view(gtis[:,1]),:]
            
        if size(gtis,1) < 1
            @warn "No GTIs longer than min_length $(min_length)"
            return mask, gtis
        end
    end   

    if dt < 0
        dt = Statistics.median(diff(times))
    end
    epsilon_times_dt = epsilon * dt

    new_gtis = [[0.0, 0.0] for _ in range(1,size(gtis,1))]
    new_gti_mask = zeros(Bool, size(gtis,1))

    gti_start = @view gtis[:, 1]
    gti_end = @view gtis[:, 2]

    for (ig,(limmin,limmax)) in enumerate(zip(gti_start,gti_end))
        limmin += safe_interval[1]
        limmax -= safe_interval[2]
        if limmax - limmin >= min_length
            new_gtis[ig][:] .= limmin, limmax
            for (i,t) in enumerate(times) 
                if (limmin + dt / 2 - epsilon_times_dt) <= t <= (limmax - dt / 2 + epsilon_times_dt)
                    mask[i] = true
                end
            end
            new_gti_mask[ig] = true
        end
    end

    return mask, mapreduce(permutedims, vcat, keepat!(new_gtis,new_gti_mask))
end

function create_gti_from_condition(time::AbstractVector{<:Real}, condition::AbstractVector{Bool};
    safe_interval::AbstractVector{<:Real}=[0,0], dt::AbstractVector{<:Real}=Float64[])
    
    if length(time) != length(condition)
        throw(ArgumentError("The length of the condition and time arrays must be the same."))
    end

    idxs = contiguous_regions(condition)

    if isempty(dt)
        dt = zero(time) .+ (time[2] .- time[1]) ./ 2
    end

    gtis = Vector{Float64}[]
    for idx in eachrow(idxs)
        startidx = idx[1]
        stopidx = idx[2] - 1

        t0 = time[startidx] - dt[startidx] + safe_interval[1]
        t1 = time[stopidx] + dt[stopidx] - safe_interval[2]
        if t1 - t0 < 0
            continue
        end
        push!(gtis,[t0, t1])
    end
    return mapreduce(permutedims, vcat, gtis)
end

function operations_on_gtis(gti_list::AbstractVector{<:AbstractMatrix{T}}, 
                            operation::Function) where {T<:Real}

    required_interval = nothing

    for gti in gti_list
        check_gtis(gti)

        combined_gti = Interval{T}[]
        for ig in eachrow(gti)
            push!(combined_gti,Interval{Closed,Open}(ig[1],ig[2]))
        end
        if isnothing(required_interval)
            required_interval = IntervalSet(combined_gti)
        else
            required_interval = operation(required_interval, IntervalSet(combined_gti))
        end
    end

    final_gti = Vector{T}[]

    for interval in required_interval.items
        push!(final_gti, [first(interval), last(interval)])
    end

    return mapreduce(permutedims, vcat, final_gti)
end

function get_btis(gtis::AbstractMatrix{<:Real})
    if isempty(gtis)
        throw(ArgumentError("Empty GTI and no valid start_time and stop_time"))
    end
    return get_btis(gtis, gtis[1,1], gtis[end,2])
end

function get_btis(gtis::AbstractMatrix{T}, start_time, stop_time) where {T<:Real}
    if isempty(gtis)
        return T[start_time stop_time]
    end
    check_gtis(gtis)

    total_interval = Interval{T, Closed, Open}[Interval{T, Closed, Open}(start_time, stop_time)]
    total_interval_set = IntervalSet(total_interval)

    gti_interval = Interval{T, Closed, Open}[]
    for ig in eachrow(gtis)
        push!(gti_interval,Interval{T, Closed, Open}(ig[1],ig[2]))
    end
    gti_interval_set = IntervalSet(gti_interval)

    bti_interval_set = setdiff(total_interval_set, gti_interval_set)

    btis = Vector{T}[]

    for interval in bti_interval_set.items
        push!(btis, [first(interval), last(interval)])
    end

    return mapreduce(permutedims, vcat, btis)
end

function time_intervals_from_gtis(gtis::AbstractMatrix{<:Real}, segment_size::Real;
                                  fraction_step::Real=1, epsilon::Real=1e-5)  
    spectrum_start_times = Float64[]

    gti_low = @view gtis[:,1]
    gti_up = @view gtis[:,2]

    for (g1,g2) in zip(gti_low,gti_up)
        if g2 - g1 + epsilon < segment_size
            continue
        end

        newtimes = range(g1, g2 - segment_size + epsilon, step = segment_size* fraction_step)
        append!(spectrum_start_times,newtimes)
    end
    return spectrum_start_times, spectrum_start_times .+ segment_size
end

function calculate_segment_bin_start(startbin::Integer, stopbin::Integer,
                                     nbin::Integer; fraction_step::Real=1)
    st = floor.(range(startbin, stopbin, step=Int(nbin * fraction_step)))
    if st[end] == stopbin
        pop!(st)
    end
    if st[end] + nbin > stopbin
        pop!(st)
    end
    return st
end

function bin_intervals_from_gtis(gtis::AbstractMatrix{<:Real}, segment_size::Real,
                                 time::AbstractVector{<:Real}; dt=nothing, 
                                 fraction_step::Real=1, epsilon::Real=0.001)
    if isnothing(dt)
        dt = Statistics.median(diff(time))
    end

    epsilon_times_dt = epsilon * dt
    nbin = round(Int, segment_size / dt)

    spectrum_start_bins = Int[]

    gti_low = @view(gtis[:, 1]) .+ (dt ./ 2 .- epsilon_times_dt)
    gti_up = @view(gtis[:, 2]) .- (dt ./ 2 .- epsilon_times_dt)

    for (g0, g1) in zip(gti_low, gti_up)
        if (g1 - g0 .+ (dt + epsilon_times_dt)) < segment_size
            continue
        end
        startbin, stopbin = searchsortedfirst.(Ref(time), [g0, g1])
        startbin -= 1
        if stopbin > length(time)
            stopbin = length(time)
        end

        if time[startbin+1] < g0
            startbin += 1
        end
        # Would be g[1] - dt/2, but stopbin is the end of an interval
        # so one has to add one bin
        if time[stopbin] > g1
            stopbin -= 1
        end

        newbins = calculate_segment_bin_start(
            startbin, stopbin, nbin, fraction_step=fraction_step)
        
        append!(spectrum_start_bins,newbins)
    end 
    return spectrum_start_bins, spectrum_start_bins.+nbin 
end

@resumable function generate_indices_of_segment_boundaries_unbinned(times::AbstractVector{<:Real},
                                                                    gti::AbstractMatrix{<:Real},
                                                                    segment_size::Real)
    start, stop = time_intervals_from_gtis(gti, segment_size)

    startidx = searchsortedfirst.(Ref(times), start)
    stopidx = searchsortedfirst.(Ref(times), stop)

    for (s, e, idx0, idx1) in zip(start, stop, startidx, stopidx)
        @yield s, e, idx0, idx1
    end
end

@resumable function generate_indices_of_segment_boundaries_binned(times::AbstractVector{<:Real},
                                                                  gti::AbstractMatrix{<:Real},
                                                                  segment_size::Real; dt=nothing)
    startidx, stopidx = bin_intervals_from_gtis(gti, segment_size, times;
                                                dt=dt)

    if isnothing(dt)
        dt = 0
    end
    for (idx0, idx1) in zip(startidx, stopidx)
        @yield times[idx0+1] - dt / 2, times[min(idx1, length(times) - 1)] - dt / 2,idx0, idx1
    end
end


generate_indices_of_segment_boundaries_binned (generic function with 1 method)

In [149]:
Pkg.add("Downloads")

   Resolving package versions...
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Project.toml`
  No Changes to `C:\Users\asus4\Desktop\julia_notebokk\Manifest.toml`


In [154]:
using Downloads, FITSIO, Plots, Test


# List of FITS file names
fits_files = ["1.fits", "2.fits", "3.fits", "4.fits", "5.fits", "6.fits", "7.fits"]

# Loop through each FITS file
for fits_file in fits_files
    println("Processing FITS file: $fits_file")

    if !isfile(fits_file)
        println("File not found: $fits_file. Skipping...")
        continue
    end

    # Step 1: Open the FITS file
    fits = FITS(fits_file, "r")  # Open in read mode

    # Step 2: List available HDUs
    println("Available HDUs in $fits_file:")
    hdu_count = length(fits)  # Number of HDUs
    for i in 1:hdu_count
        println("HDU[$i]: ", fits[i])
    end

    # Step 3: Check for image HDUs
    for i in 1:hdu_count
        hdu = fits[i]
        if hdu isa ImageHDU
            println("Image HDU found in $fits_file at HDU[$i], reading data...")
            image_data = read(hdu)
            println("Image data loaded. Shape: ", size(image_data))

            # Step 4: Plot image data
            heatmap(image_data, title="FITS Image from $fits_file - HDU[$i]", color=:viridis)
        else
            println("HDU[$i] in $fits_file is not an image.")
        end
    end

    # Step 4: Compute spectral analysis metrics
    try
        times, gti = load_gtis(fits_file)

        if isempty(gti)
            println("No GTI data found for $fits_file, skipping spectral analysis.")
        else
            segment_size = get_total_gti_length(gti)
            dt = 0.01  # Example time resolution

            avg_pds = avg_pds_from_events(times, gti, segment_size, dt, norm="frac")
            avg_cs = avg_cs_from_events(times, gti, segment_size, dt)

            fft_bins = positive_fft_bins(avg_pds)
            poisson_lvl = poisson_level(avg_pds)
            norm_pds = normalize_periodograms(avg_pds, method="leahy")
            bias = bias_term(avg_cs)
            raw_coh = raw_coherence(avg_cs, avg_pds)
            intrinsic_coh = estimate_intrinsic_coherence(avg_cs, avg_pds)
            cs_error = error_on_averaged_cross_spectrum(avg_cs)
            avg_ctrate = get_average_ctrate(times)
            flux_iterable = get_flux_iterable_from_segments(times, gti, segment_size)

            println("Computed Spectral Analysis Metrics for $fits_file")
        end
    catch e
        println("Error computing spectral metrics for $fits_file: ", e)
    end

    # Step 5: Close the FITS file
    close(fits)
    println("Finished processing $fits_file\n")
end

println("All FITS files processed successfully!")


Processing FITS file: 1.fits
Available HDUs in 1.fits:
HDU[1]: File: 1.fits
HDU: 1
Mode: read-only
Type: Image
Datatype: Float32 (physical: Int16)
Datasize: (512, 512)
Image HDU found in 1.fits at HDU[1], reading data...
512, 512)a loaded. Shape: (
CFITSIO.CFITSIOError{Nothing}(nothing, 301, "illegal HDU number", "")
Finished processing 1.fits

Processing FITS file: 2.fits
Available HDUs in 2.fits:
HDU[1]: File: 2.fits
HDU: 1
Mode: read-only
Type: Image
Datatype: Int16
Datasize: (512, 512)
File: 2.fits
HDU: 2 (name=LS3)
Type: Table
Rows: 4
Columns: Name      Size  Type     TFORM  
         SRC_NR          Int16    1I     
         LEV1_X          Float32  1E     
         LEV1_Y          Float32  1E     
         NET_CTS         Float32  1E     
         BKG_CTS         Float32  1E     
         S_LIKE          Float32  1E     
         C_SIZE          Int16    1I     
         NN_DIST         Float32  1E     
         RIB_DIST        Float32  1E     
HDU[3]: File: 2.fits
HDU: 3 (name=